In [1]:
import pandas as pd
import numpy as np
import re
import json
import csv
import threading
import os

## Read in WSB json

In [7]:
with open('wsbData.json', 'r') as f:
    data = f.readlines()

# remove the trailing "\n" from each line
data = map(lambda x: x.rstrip(), data)

# each element of 'data' is an individual JSON object.
# i want to convert it into an *array* of JSON objects
# which, in and of itself, is one large JSON object
# basically... add square brackets to the beginning
# and end, and have all the individual business JSON objects
# separated by a comma
data_json_str = "[" + ','.join(data) + "]"

# now, load it into pandas
df = pd.read_json(data_json_str)

## Read in Company Tickers

In [12]:
df_company = pd.read_csv("companylist.csv")
df_company.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,YI,"111, Inc.",6.00,4.790733e+07,7984555.0,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi,NaN
1,PIH,"1347 Property Insurance Holdings, Inc.",3.89,2.331056e+07,NaN,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih,NaN
2,PIHPP,"1347 Property Insurance Holdings, Inc.",16.70,0.000000e+00,NaN,NaN,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pihpp,NaN
3,TURN,180 Degree Capital Corp.,1.83,5.695246e+07,NaN,NaN,Finance,Finance/Investors Services,https://www.nasdaq.com/symbol/turn,NaN
4,FLWS,"1-800 FLOWERS.COM, Inc.",10.99,7.053203e+08,NaN,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws,NaN


In [15]:
tickers = df_company["Symbol"]
tickers.head()

0       YI
1      PIH
2    PIHPP
3     TURN
4     FLWS
Name: Symbol, dtype: object

## Extract Tickers

In [22]:
def extract_tickers(string):
    e = re.findall('[A-Z]{1,4}|\d{1,3}(?=\.)|\d{4,}', string)
    return list(set(e).intersection(set(tickers.values)))

In [26]:
df["tickers"] = df["body"].apply(extract_tickers)

In [32]:
df["datetime"] = pd.to_datetime(df["created_utc"],unit='s')

In [33]:
df.head()

,archived,author,author_flair_css_class,author_flair_text,body,controversiality,created_utc,distinguished,downs,gilded,...,name,parent_id,retrieved_on,score,score_hidden,subreddit,subreddit_id,ups,tickers,datetime
0,1.0,[deleted],NaN,NaN,[deleted],0,1400098596,NaN,0.0,0,...,t1_chi0ufo,t1_chhzelx,1.433797e+09,0,0.0,wallstreetbets,t5_2th52,0.0,[],2014-05-14 20:16:36
1,0.0,JamesAQuintero,NaN,NaN,"Lol. Yeah, Welp.",0,1412888184,NaN,0.0,0,...,t1_cl52foo,t1_cl47s92,1.426605e+09,1,0.0,wallstreetbets,t5_2th52,1.0,[],2014-10-09 20:56:24
2,1.0,[deleted],NaN,NaN,[deleted],0,1398860210,NaN,0.0,0,...,t1_ch5wdhy,t1_ch5diku,1.433588e+09,1,0.0,wallstreetbets,t5_2th52,1.0,[],2014-04-30 12:16:50
3,1.0,Zytran,NaN,NaN,Crypto miners is not a significant enough mark...,0,1392824901,NaN,0.0,0,...,t1_cfj9fzo,t3_1y409u,1.432466e+09,1,0.0,wallstreetbets,t5_2th52,1.0,[AMD],2014-02-19 15:48:21
4,1.0,FarmPal,NaN,NaN,Sold a covered call at 560. I almost wish it d...,0,1398311440,NaN,0.0,0,...,t1_ch0jw88,t3_23sjde,1.433495e+09,1,0.0,wallstreetbets,t5_2th52,1.0,[],2014-04-24 03:50:40


In [3]:
import yfinance as yf
import yahoofinancials